In [1]:
import pandas as pd

In [2]:
games = pd.read_csv('nba_games/games.csv')

games = games[games['SEASON'] > 2020]
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])

for idx, row in games.iterrows():
    games.loc[idx, 'DAY'] = row['GAME_DATE_EST'].timetuple().tm_yday
    games.loc[idx, 'WEEK'] = row['GAME_DATE_EST'].isocalendar().week
    games.loc[idx, 'MONTH'] = row['GAME_DATE_EST'].month


In [3]:
game_details = pd.read_csv('nba_games/games_details.csv')
df_home = pd.merge(game_details, games, left_on=["GAME_ID", "TEAM_ID"], right_on=['GAME_ID', "HOME_TEAM_ID"], how="inner")
df_visit = pd.merge(game_details, games, left_on=["GAME_ID", "TEAM_ID"], right_on=['GAME_ID', "VISITOR_TEAM_ID"], how="inner")

/var/folders/j4/xht4swnx3v38mbt0w75bjqs40000gn/T/ipykernel_69004/3113203320.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  game_details = pd.read_csv('nba_games/games_details.csv')


Check for nulls and find minimum number of players per team, minimum number of players to start a NBA game is 9

In [4]:
player_teams_df = df_home.groupby(['GAME_ID', 'TEAM_ID']).agg({'PLAYER_ID': ['count']})
print(player_teams_df[player_teams_df.isnull().any(axis=1)])
print(player_teams_df.min())

player_teams_df = df_visit.groupby(['GAME_ID', 'TEAM_ID']).agg({'PLAYER_ID': ['count']})
print(player_teams_df[player_teams_df.isnull().any(axis=1)])
print(player_teams_df.min())


Empty DataFrame
Columns: [(PLAYER_ID, count)]
Index: []
PLAYER_ID  count    10
dtype: int64
Empty DataFrame
Columns: [(PLAYER_ID, count)]
Index: []
PLAYER_ID  count    9
dtype: int64


In [5]:
def calculate_seconds_played(df):
    for idx, row in df.iterrows():
        minutes = row['MIN']
        seconds = 0
        if (isinstance(minutes, str)):
            m, s = minutes.split(":")
            seconds = int(float(m) * 60 + float(s))
        df.loc[idx, 'SEC'] = seconds
    return df

df_home = calculate_seconds_played(df_home)
df_visit = calculate_seconds_played(df_visit)


In [6]:
df_home = df_home[['GAME_ID', 'HOME_TEAM_ID', 'PLAYER_ID', 'SEC']]
df_visit = df_visit[['GAME_ID', 'VISITOR_TEAM_ID', 'PLAYER_ID', 'SEC']]


In [7]:
def pivot_players(df, home):
    gb = df.groupby(['GAME_ID', f"{home}_TEAM_ID"])
    game_teams = [gb.get_group(x) for x in gb.groups]
    
    df_pivoted = pd.DataFrame()
    
    for x in game_teams:
        tmp = x.nlargest(9, 'SEC')
        num = 1
        for idx, row in tmp.iterrows():
            tmp.loc[idx, 'SEC'] = f"{home}_P{num}"
            num += 1
            
        tmp.rename(columns={'SEC':'NUM'}, inplace=True)
        tmp = pd.pivot_table(tmp, index=['GAME_ID', f"{home}_TEAM_ID"], columns='NUM', values='PLAYER_ID', aggfunc= lambda r:r)
        df_pivoted = pd.concat([df_pivoted, tmp])
    
    return df_pivoted
    
df_home_pivoted = pivot_players(df_home, "HOME")
df_visit_pivoted = pivot_players(df_visit, "VISITOR")

In [ ]:
df_visit_pivoted.to_csv("visit_pivoted.csv")
df_home_pivoted.to_csv("home_pivoted.csv")

In [ ]:
home = pd.read_csv("home_pivoted.csv")
visit = pd.read_csv("visit_pivoted.csv")
games_filter = games[['GAME_ID', 'DAY', 'WEEK', 'MONTH', 'HOME_TEAM_WINS']]

df_data = pd.merge(home, visit, on=['GAME_ID'], how='outer')
df_data = pd.merge(df_data, games_filter, on='GAME_ID')

In [ ]:
df_data.to_csv("processed.csv")